In [59]:
from collections import Counter

def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir, f) for f in os.listdir(train_dir)]
    all_words = []
    for mail in emails:
        with open(mail) as m:
            for i, line in enumerate(m):
                if i == 2:
                    words = line.split()
                    all_words += words
    
    dictionary = Counter(all_words)

    list_to_remove = list(dictionary.keys())
    for item in list_to_remove:
        if item.isalpha() == False:
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
    dictionary = dictionary.most_common(3000)

    return dictionary

In [60]:
import numpy as np

def extract_features(mail_dir):
    files = [os.path.join(mail_dir, fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files), 3000))
    docID = 0
    for fil in files:
        with open(fil) as fi:
            for i, line in enumerate(fi):
                if i == 2:
                    words = line.split()
                    for word in words:
                        wordID = 0
                        for i, d in enumerate(dictionary):
                            if d[0] == word:
                                wordID = i
                                features_matrix[docID, wordID] = words.count(word)
            docID = docID + 1
    return features_matrix

In [61]:
dictionary = make_Dictionary('./ling-spam/train-mails/')
train_feature = extract_features('./ling-spam/train-mails')
ham_feature, spam_feature = np.split(train_feature, 2)
test_feature = extract_features('./ling-spam/test-mails')

In [62]:
def calculate(feature_matrix):
    prob = sum(feature_matrix) / feature_matrix.sum()
    for i in range(3000):
        if not prob[i]:
            prob[i] = 1
    min_prob = min(prob)
    for i in range(3000):
        if prob[i] == 1:
            prob[i] = min_prob
    return prob

ham_likelihood = calculate(ham_feature)
spam_likelihood = calculate(spam_feature)

In [63]:
def naive_Bayes(test, ham, spam):
    n = len(test)
    res = [False for i in range(n)]
    prob_ham, prob_spam = np.zeros(n), np.zeros(n)
    for i in range(n):
        for j in range(3000):
            if test[i, j] > 0:
                prob_ham[i] += np.log(ham[j]) * test[i, j]
                prob_spam[i] += np.log(spam[j]) * test[i, j]
        if prob_ham[i] < prob_spam[i]:
            res[i] = True
    return res

result = naive_Bayes(test_feature, ham_likelihood, spam_likelihood)

In [64]:
n = len(test_feature)
mid = n // 2
TP = sum(result[mid:n])
FP = sum(result[0:mid])
TN = n / 2 - FP
FN = n / 2 - TP
acc = (TP + TN) / n
prec = TP / (TP + FP)
rec = TP / (TP + FN)
f_1 = 2 * (rec * prec) / (rec + prec)
print(f'Accuracy: {acc}\nPrecision: {prec}\nRecall: {rec}\nF-1 score: {f_1}')

Accuracy: 0.9615384615384616
Precision: 0.9918032786885246
Recall: 0.9307692307692308
F-1 score: 0.9603174603174605
